# 패키지 설치
1. langchain
2. faiss-cpu
3. Chroma
4. python-docx
5. dotenv
6. openai

In [119]:
!pip install langchain langchain-openai langchain-chroma langchain-community openai faiss-cpu python-docx dotenv

  Using cached chromadb-1.0.15-cp39-abi3-win_amd64.whl.metadata (7.1 kB)
  Using cached build-1.2.2.post1-py3-none-any.whl.metadata (6.5 kB)
  Using cached pybase64-1.4.2-cp312-cp312-win_amd64.whl.metadata (9.0 kB)
  Using cached uvicorn-0.35.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached posthog-5.4.0-py3-none-any.whl.metadata (5.7 kB)
  Using cached onnxruntime-1.22.1-cp312-cp312-win_amd64.whl.metadata (5.1 kB)
  Using cached opentelemetry_api-1.35.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.35.0-py3-none-any.whl.metadata (2.4 kB)
  Using cached opentelemetry_sdk-1.35.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached pypika-0.48.9-py2.py3-none-any.whl
  Using cached overrides-7.7.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached importlib_resources-6.5.2-py3-none-any.whl.metadata (3.9 kB)
  Using cached bcrypt-4.3.0-cp39-abi3-win_amd64.whl.metadata (10 kB)
  Using cached typer-0.16.0-py3-none-any.whl.metadata (15 kB)
  Using c

## 환경 변수 설정

In [109]:
from dotenv import load_dotenv
import os
load_dotenv()

file_path = os.getenv("FILE_PATH")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_BASE"] = os.getenv("OPENAI_API_BASE")

print(f'file_path: {file_path}')


file_path: Structural_Interpretation.docx


## 문서 로딩
- docx 문서 로딩

In [113]:
from docx import Document as DocxDocument

file_path = "Structural_Interpretation.docx"

def load_docx(file_path) -> str:
    try:
        """
        문서 파일 읽고 텍스트로 변환
        """
        doc = DocxDocument(file_path)
        text = []
        for para in doc.paragraphs:
            text.append(para.text)
        
        print(f"loaded : {file_path}")

        return "\n".join(text)
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return ""

full_text = load_docx(file_path)


loaded : Structural_Interpretation.docx


# 데이터 청킹
### 문서 구조
- [] 대괄호 category
- {} 중괄호 소분류 

해당 분류 기준을 중심으로 chunking
## WHY ? 
1. 분류를 메타데이터로 정확도 향상
2. 의미기반 chunking 으로 노이즈 없는 store구성을 위해서

In [117]:
# 문서 저장소
# 문서는 [] 로 대주제, {}로 소주제로 나뉘어져 있음 기준으로 청킹
import re
from langchain_core.documents import Document
from pprint import pprint

chunks = [] # 청크 저장소
main_topic = None # 대주제
sub_topic = None # 소주제
content = [] # 본문

def add_chunk(main, sub, content):
    """
    청크를 추가하는 함수
    """
    if main and content:
        chunks.append(
            Document(
                page_content=" ".join(content).strip(),
                metadata={"main_topic": main, "sub_topic": sub},
            )
        )

# 청킹
"""
각 주제가 바뀌는 [], {} 단위로 청크를 나누고 저장
문서 구조) 
[대주제1]
{소주제1}
내용1
내용2
{소주제2}
내용3

"""
try:
    for line in full_text.split("\n"):
        line = line.strip()
        # 빈줄
        if not line:
            continue
        # 대주제
        if re.match(r"^\[.*\]$", line):
            # 이전 청크 저장
            add_chunk(main_topic, sub_topic, content)

            content = []
            main_topic = line[1:-1].strip()
            sub_topic = None
            continue
        # 소주제
        if re.match(r"^\{.*\}$", line):
            # 이전 청크 저장
            add_chunk(main_topic, sub_topic, content)
            content = []
            sub_topic = line[1:-1].strip()
            continue
        # 내용
        content.append(line)
    # 마지막 청크 저장
    add_chunk(main_topic, sub_topic, content)
    print(f"Total chunks: {len(chunks)}")
    print("✅ done")
except Exception as e:
    print(f"Error during chunking: {e}")


Total chunks: 77
✅ done


# 청크 임베딩
## vectorDB
### FAISS vs ChromaDB
---
### 성능 평가 결과
:
---

## 임베딩 모델 설정
### open-ai vs gemini
### text-embedding-3-small vs text-embedding-004
---
### 성능 평가 결과
:

In [118]:
import faiss
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

# 임베딩모델 설정
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
)


In [ ]:
# %%time
# 벡터 저장소 로드
import time

batch_size = 10  # 한번에 처리할 chunk 개수 (필요시 조절)
all_chunks = chunks  # chunks는 이미 정의되어 있는 리스트
faiss_store = None

for i in range(0, len(all_chunks), batch_size):
    batch = all_chunks[i:i + batch_size]
    # 벡터스토어 생성 및 추가
    if faiss_store is None:
        faiss_store = FAISS.from_documents(batch, embedding=embeddings)
    else:
        faiss_store.add_documents(batch)
    # 요청 속도 제한에 대비하여 딜레이 추가
    time.sleep(1)

### TEST

In [ ]:
for doc in faiss_store.similarity_search("오른쪽으로 치우쳐짐", k=4):
    print(doc.page_content, doc.metadata)

# for doc in chunks:
#     print(doc.page_content, doc.metadata)

In [ ]:
faiss_store.save_local("faiss_store")

# 평가용 GT 생성

In [101]:
# 각 청크에서 Q&A 형식의 GroundTruth 생성
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field

# output parser 설정
class GroundTrunth(BaseModel):
    Q: str = Field(description="질문: 짧고 명확한 사실에 대한 형태의 쿼리 (예: 그림을 그리는 시간이 짧았다.) ")
    A: str = Field(description="답변: context를 읽고 풍부하고 전문가적인 응답을 생성해야 합니다. 단순히 문맥 context를 읽는 것이 아닌 전문가가 해당 특징에 대해 서술하듯이 이야기 해야합니다. context와 해석 여지를 제공하여 평가하기에 용이한 응답을 생성하세요.절대 거짓이나 모호한 응답을 생성해서는 안됩니다. context를 이해하고 작성해주세요.")

parser = JsonOutputParser(pydantic_object=GroundTrunth)

# 프롬프트 템플릿: 각 청크에서 질문-응답 쌍을 생성하도록 명시
prompt = PromptTemplate(
    template="""
사전 정의 : 당신은 HTP(Home, tree, person) 심리 검사에 대한 전문가입니다. 아래에 주어지는 문서를 확인하고 RAG를 구축하고 평가하기 위한 GroundTruth를 생성해야 합니다.
구축하고자 하는 RAG 파이프라인은 상담사가 그림을 관찰하고 관찰한 그림에 대한 사실적 질문을 생성하여 그에 대한 해석 결과를 제공하는 것입니다.
(예시)
예를 들어 상담사는 그림을 관찰하고 "집이 오른쪽으로 기울어져 있다. 문이 집의 크기에 비해 작게 그려져 있고, 지붕은 존재하고 있으나 굴뚝을 그리지 않았다." 라고 작성할 수 있습니다.
그러면 질문을 "집이 오른쪽으로 기울어져 있다", "문이 집의 크기에 비해 작다", "굴뚝이 없다" 와 같이 각 특이사항 별로 쿼리를 분해하고 RAG에서 검색을 할 수 있도록 합니다.
각 쿼리에 대한 해석을 정리, 종합하여 LLM 에게 전달하고 LLM은 객관적인 RAG 기반 응답을 생성합니다.

예시와 같은 서비스를 구축하기 위해 RAG 평가를 위한 GroundTruth를 생성해야하는게 당신의 업무입니다.
(질문)
질문을 생성할 때에는 마치 그림을 보고 관찰한 결과를 보고하듯 작성해야 합니다. 가상의 그림이 있다고 생각하고 작성하세요.
즉 context의 내용으로만 질문을 작성하지 말고 마치 그림이 있다고 생각하고 관찰한 결과를 작성해야 합니다.
(응답)
질문에 대한 응답을 생성할 때에는 context를 읽고 풍부하고 전문가적인 응답을 생성해야 합니다. 단순히 문맥 context를 읽는 것이 아닌 전문가가 해당 특징에 대해 서술하듯이 이야기 해야합니다.
context와 해석 여지를 제공하여 평가하기에 용이한 응답을 생성하세요. 절대 거짓이나 모호한 응답을 생성해서는 안됩니다. context를 이해하고 작성해주세요.

아래의 문서 내용을 바탕으로, 의문문이 아닌 팩트 기반의 진술문을 쿼리와 답변 쌍을 하나만 생성하세요. 
\n{format_instructions}\n

문서 내용:
\n{context}\n

""",
    input_variables=["context"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)

# chain 생성
chain = prompt | llm | parser



# 각 청크당 GT 생성

In [103]:
ground_truths = []

try:
    for idx, doc in enumerate(chunks):
        context = doc.page_content
        result = chain.invoke({"context": context})
        ground_truths.append({
            "chunk_id": idx,
            "main_topic": doc.metadata.get("main_topic"),
            "sub_topic": doc.metadata.get("sub_topic"),
            "qa_pairs": result
        })
        print(f"✅Processed chunk {idx + 1}/{len(chunks)}")
except Exception as e:
    print(f"❌문제 발생 : {e}")

# 결과 예시 출력
pprint(ground_truths[:5])

✅Processed chunk 1/86
✅Processed chunk 2/86
✅Processed chunk 3/86
✅Processed chunk 4/86
✅Processed chunk 5/86
✅Processed chunk 6/86
✅Processed chunk 7/86
✅Processed chunk 8/86
✅Processed chunk 9/86
✅Processed chunk 10/86
✅Processed chunk 11/86
✅Processed chunk 12/86
✅Processed chunk 13/86
✅Processed chunk 14/86
✅Processed chunk 15/86
✅Processed chunk 16/86
✅Processed chunk 17/86
✅Processed chunk 18/86
✅Processed chunk 19/86
✅Processed chunk 20/86
✅Processed chunk 21/86
✅Processed chunk 22/86
✅Processed chunk 23/86
✅Processed chunk 24/86
✅Processed chunk 25/86
✅Processed chunk 26/86
✅Processed chunk 27/86
✅Processed chunk 28/86
✅Processed chunk 29/86
✅Processed chunk 30/86
✅Processed chunk 31/86
✅Processed chunk 32/86
✅Processed chunk 33/86
✅Processed chunk 34/86
✅Processed chunk 35/86
✅Processed chunk 36/86
✅Processed chunk 37/86
✅Processed chunk 38/86
✅Processed chunk 39/86
✅Processed chunk 40/86
✅Processed chunk 41/86
✅Processed chunk 42/86
✅Processed chunk 43/86
✅Processed chunk 44/

# GT 저장

In [104]:
import json

# ground_truths를 JSON 파일로 저장
with open("ground_truths.json", "w", encoding="utf-8") as f:
    json.dump(ground_truths, f, ensure_ascii=False, indent=2)

print("✅ ground_truths.json 파일로 저장 완료")

✅ ground_truths.json 파일로 저장 완료
